In [1]:
import sys
from pathlib import Path
import pandas as pd

# Determine the root directory (adjust the number of parents as needed)
root_dir = Path.cwd().parents[0]  # Adjust based on your folder structure

# Add the root directory to sys.path
sys.path.append(str(root_dir))
print(sys.path)

from models.naive_bayes import NaiveBayes




['c:\\Users\\cocol\\Documents\\Cours\\3A\\PIC\\defensive\\notebooks', 'c:\\ProgramData\\anaconda3\\python311.zip', 'c:\\ProgramData\\anaconda3\\DLLs', 'c:\\ProgramData\\anaconda3\\Lib', 'c:\\ProgramData\\anaconda3', '', 'C:\\Users\\cocol\\AppData\\Roaming\\Python\\Python311\\site-packages', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages\\vboxapi-1.0-py3.11.egg', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages\\win32', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages\\win32\\lib', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\cocol\\Documents\\Cours\\3A\\PIC\\defensive']


In [24]:
from datasets import load_dataset

dataset = load_dataset("TrainingDataPro/email-spam-classification")

In [51]:
dataset_df = pd.read_csv("../emails.csv", delimiter=",")
dataset_df.rename(columns={'text': 'body', 'label': 'ground_truth'}, inplace=True)

dataset_df = dataset_df.dropna()
dataset_df

,ground_truth,body
0,1,ounce feather bowl hummingbird opec moment ala...
1,1,wulvob get your medircations online qnb ikud v...
2,0,computer connection from cnn com wednesday es...
3,1,university degree obtain a prosperous future m...
4,0,thanks for all your answers guys i know i shou...
...,...,...
83443,0,hi given a date how do i get the last date of ...
83444,1,now you can order software on cd or download i...
83445,1,dear valued member canadianpharmacy provides a...
83446,0,subscribe change profile contact us long term ...


In [3]:
dataset_df = pd.DataFrame(dataset['train'])
dataset_df.rename(columns={'title': 'subject', 'text': 'body'}, inplace=True)

dataset_df['ground_truth'] = dataset_df['type'].apply(lambda x: 1 if x == 'spam' else 0)

dataset_df.drop(columns=['type'], inplace=True)
dataset_df

,subject,body,ground_truth
0,?? the secrets to SUCCESS,"Hi James,\n\nHave you claim your complimentary...",1
1,?? You Earned 500 GCLoot Points,"\nalt_text\nCongratulations, you just earned\n...",0
2,?? Your GitHub launch code,"Here's your GitHub launch code, @Mortyj420!\n ...",0
3,[The Virtual Reward Center] Re: ** Clarifications,"Hello,\n \nThank you for contacting the Virtua...",0
4,"10-1 MLB Expert Inside, Plus Everything You Ne...","Hey Prachanda Rawal,\n\nToday's newsletter is ...",1
...,...,...,...
79,Your application for the position of Child Pr...,"Dear Maryam, \n\n \n\nI would like to thank yo...",0
80,Your Kilimall Account is Ready - Shopping Now!,"Dear Customer,\n\nWelcome to Kilimall, Thanks ...",0
81,Your Steam account: Access from new web or mob...,"Dear vladis163rus,\nHere is the Steam Guard co...",0
82,Your uploaded document is rejected,View In Browser | Log in\n \n \n\nSkrill logo\...,0


In [80]:
classifier1 = NaiveBayes(dataset_df, type='bert')
classifier1.train(dataset_df)

classifier2 = NaiveBayes(dataset_df, type='sklearn')
classifier2.train(dataset_df)

Accuracy: 0.547813061713601 0.547813061713601
Accuracy: 0.9757938885560216 0.9757938885560216


0.9757938885560216

In [52]:
import re
import nltk
from nltk.corpus import stopwords

def clean_email(text):
    text = text.lower()
    text = re.sub(r"subject:.*", "", text)  # Supprimer l’en-tête "Subject:"
    text = re.sub(r"<.*?>", "", text)  # Supprimer le HTML
    text = re.sub(r"https?://\S+|www\.\S+", "", text)  # Supprimer les URLs
    text = re.sub(r"[^\w\s]", "", text)  # Supprimer la ponctuation
    text = re.sub(r"\d+", "", text)  # Supprimer les chiffres
    text = re.sub(r"\n", " ", text) # Supprimer les sauts de ligne
    return " ".join(word for word in text.split() if word not in stop_words)  # Supprimer les stopwords
    return text

# Télécharger les stopwords
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cocol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [57]:
dataset_df['body'] = dataset_df.apply(lambda x: clean_email(x['body']), axis=1)

In [58]:
dataset_df[['address', 'subject', 'domain', 'domain_extension']] = None
dataset_df['subject'] = " "

In [59]:
dataset_df

,ground_truth,body,subject,address,domain,domain_extension
0,1,ounce feather bowl hummingbird opec moment ala...,,None,None,None
1,1,wulvob get medircations online qnb ikud viagra...,,None,None,None
2,0,computer connection cnn com wednesday escapenu...,,None,None,None
3,1,university degree obtain prosperous future mon...,,None,None,None
4,0,thanks answers guys know checked rsync manual ...,,None,None,None
...,...,...,...,...,...,...
83443,0,hi given date get last date month data form yy...,,None,None,None
83444,1,order software cd download site immediately de...,,None,None,None
83445,1,dear valued member canadianpharmacy provides w...,,None,None,None
83446,0,subscribe change profile contact us long term ...,,None,None,None


In [18]:
# Assuming classify method returns a tuple where the second element is the classification result
result1 = classifier1.classify(dataset_df)[1]
result2 = classifier2.classify(dataset_df)[1]

s = 0
for r1, r2 in zip(result1, result2):
    s += r1 and not r2 or not r1 and r2
s

17

In [79]:
import time
from transformers import BertTokenizer

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Measure the time taken to encode
start_time = time.time()
encoded_input = tokenizer(dataset_df['body'][13564], return_tensors='np', truncation=True, max_length=10000, padding="max_length")
end_time = time.time()

print(f"Time taken to encode: {end_time - start_time} seconds")
print(f"{(end_time - start_time) * 83000}")

Time taken to encode: 0.009706974029541016 seconds
805.6788444519043
